In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks')

!pip install keras-tuner
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 12.4 MB/s eta 0:00:00


In [3]:
import importlib
createTriplet = importlib.import_module('createTriplet')

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import gc
import kerastuner as kt
import cv2
import createTriplet
from createTriplet import augment_images, create_triplets
import optuna


# Import the necessary modules for pre-trained model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Triplet loss function
def triplet_loss(y_true, y_pred, alpha=0.2):
    embedding_size = y_pred.shape[-1] // 3
    anchor, positive, negative = y_pred[:, :embedding_size], y_pred[:, embedding_size:2*embedding_size], y_pred[:, 2*embedding_size:]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + alpha, 0.0)
    return tf.reduce_mean(loss)

# Create embedding model with EfficientNetB0
def create_embedding_model(input_shape, l2_strength=0.01):
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation="relu", kernel_regularizer=regularizers.l2(l2_strength))(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(l2_strength))(x)
    return Model(inputs=base_model.input, outputs=x)

# Load images
path_to_directory = "/content/drive/MyDrive/Datasets/One_images"
image_paths = [os.path.join(path_to_directory, file) for file in os.listdir(path_to_directory) if file.endswith(('.png', '.jpg', '.jpeg'))]

# Create a list of images and their corresponding class labels
images = []
labels = []
num_augmented_images = 6
for image_path in image_paths:
    augmented_images = augment_images(image_path, num_augmented_images=num_augmented_images)
    class_name = os.path.basename(image_path).split('_')[0]  # Extract class name from the image path
    images.extend(augmented_images)
    labels.extend([class_name] * len(augmented_images))

# Load and preprocess image
def load_image(image_data, target_size=(224, 224)):
    if isinstance(image_data, str):
        img = img_to_array(load_img(image_data, target_size=target_size))
    elif isinstance(image_data, np.ndarray):
        img = cv2.resize(image_data, target_size)
    else:
        raise TypeError(f"image_data should be a string or a numpy array, not {type(image_data)}")
    
    return preprocess_input(img)

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, train_size=0.8, stratify=labels)

# Free up memory
del images
del labels
gc.collect()

train_anchor_images, train_positive_images, train_negative_images = create_triplets(train_images, train_labels)
val_anchor_images, val_positive_images, val_negative_images = create_triplets(val_images, val_labels)

# Free up memory
del train_images
del val_images
del train_labels
del val_labels
gc.collect()

loaded_train_anchor_images = np.array([load_image(img) for img in train_anchor_images])
loaded_train_positive_images = np.array([load_image(img) for img in train_positive_images])
loaded_train_negative_images = np.array([load_image(img) for img in train_negative_images])

# Load the images for the validation triplets
loaded_val_anchor_images = np.array([load_image(img) for img in val_anchor_images])
loaded_val_positive_images = np.array([load_image(img) for img in val_positive_images])
loaded_val_negative_images = np.array([load_image(img) for img in val_negative_images])

num_train_triplets = len(train_anchor_images)
num_val_triplets = len(val_anchor_images)

# Free up memory
del train_anchor_images
del train_positive_images
del train_negative_images
del val_anchor_images
del val_positive_images
del val_negative_images
gc.collect()

def model_builder(trial):
    # Create the base network for embeddings
    input_shape = (224, 224, 3)

    # Hyperparameters
    l2_strength = trial.suggest_loguniform("l2_strength", 1e-6, 1e-2)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)
    dense_units = trial.suggest_categorical("dense_units", [128, 256, 512])

    embedding_model = create_embedding_model(input_shape, l2_strength=l2_strength)

    # Add dropout layers to the embedding model
    x = embedding_model.output
    for i in range(trial.suggest_int("num_dropout_layers", 1, 3)):
        x = Dropout(rate=dropout_rate, name=f"dropout_layer_{i+1}")(x)
    x = Dense(dense_units, activation="relu", kernel_regularizer=regularizers.l2(l2_strength), name="dense_layer")(x)
    modified_embedding_model = Model(inputs=embedding_model.input, outputs=x)

    # Create the Siamese network with triplet loss
    anchor_input = Input(shape=input_shape, name="anchor_input")
    positive_input = Input(shape=input_shape, name="positive_input")
    negative_input = Input(shape=input_shape, name="negative_input")

    anchor_embedding = modified_embedding_model(anchor_input)
    positive_embedding = modified_embedding_model(positive_input)
    negative_embedding = modified_embedding_model(negative_input)

    outputs = tf.keras.layers.concatenate([anchor_embedding, positive_embedding, negative_embedding], axis=-1)
    siamese_triplet_model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=outputs)

    # Learning Rate
    learning_rate = trial.suggest_categorical("learning_rate", [1e-2, 1e-3, 1e-4])

    # Compile the model
    siamese_triplet_model.compile(optimizer=Adam(learning_rate), loss=triplet_loss)

    return siamese_triplet_model

def objective(trial):
    model = model_builder(trial)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        [loaded_train_anchor_images, loaded_train_positive_images, loaded_train_negative_images],
        np.zeros(num_train_triplets),
        validation_data=([loaded_val_anchor_images, loaded_val_positive_images, loaded_val_negative_images], np.zeros(num_val_triplets)),
        batch_size=32,
        epochs=20,
        callbacks=[early_stopping]
    )

    loss = model.evaluate([loaded_val_anchor_images, loaded_val_positive_images, loaded_val_negative_images], np.zeros(num_val_triplets), verbose=0)
    return loss


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

# Get the best hyperparameters
best_hyperparameters = study.best_trial.params
print("Best hyperparameters: ", best_hyperparameters)

# Build the best model with the best_hyperparameters and train it
best_model = model_builder(optuna.trial.FixedTrial(best_hyperparameters))



# Train the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

best_model.fit(
    [loaded_train_anchor_images, loaded_train_positive_images, loaded_train_negative_images],
    np.zeros(num_train_triplets),
    validation_data=([loaded_val_anchor_images, loaded_val_positive_images, loaded_val_negative_images], np.zeros(num_val_triplets)),
    batch_size=32,
    epochs=20,
    callbacks=[early_stopping]  # Pass the EarlyStopping callback to the fit method
)

# Get the best embedding model from the best Siamese network
best_embedding_model = Model(inputs=best_model.get_layer(index=2).get_input_at(0), outputs=best_model.get_layer(index=2).get_output_at(0))

# Save the best embedding model
tf.saved_model.save(best_model, "best_siamese_model")




[I 2023-03-31 15:40:40,762] A new study created in memory with name: no-name-e04119e6-d717-4115-9296-3a44a70e5387
<ipython-input-11-1d8287abab42>:115: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  l2_strength = trial.suggest_loguniform("l2_strength", 1e-6, 1e-2)
<ipython-input-11-1d8287abab42>:116: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)


Epoch 1/20
32/32 [==============================] - 71s 429ms/step - loss: 39.6053 - val_loss: 10.4080
Epoch 2/20
32/32 [==============================] - 8s 257ms/step - loss: 16.4149 - val_loss: 10.4071
Epoch 3/20
32/32 [==============================] - 8s 255ms/step - loss: 11.8524 - val_loss: 10.3844
Epoch 4/20
32/32 [==============================] - 8s 255ms/step - loss: 10.6795 - val_loss: 10.3526
Epoch 5/20
32/32 [==============================] - 8s 255ms/step - loss: 10.4429 - val_loss: 10.3161
Epoch 6/20
32/32 [==============================] - 8s 254ms/step - loss: 10.3562 - val_loss: 10.2748
Epoch 7/20
32/32 [==============================] - 8s 250ms/step - loss: 10.3106 - val_loss: 1721592192.0000
Epoch 8/20
32/32 [==============================] - 8s 252ms/step - loss: 10.2373 - val_loss: 1879143936.0000
Epoch 9/20
32/32 [==============================] - 8s 255ms/step - loss: 10.1831 - val_loss: 512443.1875


[I 2023-03-31 15:43:05,232] Trial 0 finished with value: 10.274825096130371 and parameters: {'l2_strength': 0.007306300598138226, 'dropout_rate': 0.17107117079162043, 'dense_units': 256, 'num_dropout_layers': 3, 'learning_rate': 0.0001}. Best is trial 0 with value: 10.274825096130371.


Epoch 1/20
32/32 [==============================] - 74s 542ms/step - loss: 194.7104 - val_loss: 2.6410
Epoch 2/20
32/32 [==============================] - 8s 250ms/step - loss: 45.2518 - val_loss: 2.6448
Epoch 3/20
32/32 [==============================] - 8s 251ms/step - loss: 9.3430 - val_loss: 2.6444
Epoch 4/20
32/32 [==============================] - 8s 258ms/step - loss: 4.1735 - val_loss: 2.6433


[I 2023-03-31 15:44:51,966] Trial 1 finished with value: 2.641005039215088 and parameters: {'l2_strength': 0.001704025348463876, 'dropout_rate': 0.47723534023381864, 'dense_units': 512, 'num_dropout_layers': 3, 'learning_rate': 0.0001}. Best is trial 1 with value: 2.641005039215088.


...

Epoch 1/20
32/32 [==============================] - 70s 453ms/step - loss: 14016.4932 - val_loss: 24185.0957
Epoch 2/20
32/32 [==============================] - 8s 256ms/step - loss: 165.5417 - val_loss: 12.7789
Epoch 3/20
32/32 [==============================] - 8s 253ms/step - loss: 21.3782 - val_loss: 4.1634
Epoch 4/20
32/32 [==============================] - 8s 256ms/step - loss: 10.8622 - val_loss: 4.1619
Epoch 5/20
32/32 [==============================] - 8s 254ms/step - loss: 7.0494 - val_loss: 4.1611
Epoch 6/20
32/32 [==============================] - 8s 254ms/step - loss: 5.8883 - val_loss: 4.1599
Epoch 7/20
32/32 [==============================] - 8s 255ms/step - loss: 5.4308 - val_loss: 4.1589
Epoch 8/20
32/32 [==============================] - 8s 255ms/step - loss: 5.3633 - val_loss: 4.1576
Epoch 9/20
32/32 [==============================] - 8s 254ms/step - loss: 5.0514 - val_loss: 4.1565
Epoch 10/20
32/32 [==============================] - 8s 254ms/step - loss: 5.1409 - va

[I 2023-03-31 15:50:26,829] Trial 3 finished with value: 4.138205528259277 and parameters: {'l2_strength': 3.673750863784056e-05, 'dropout_rate': 0.13429297384147398, 'dense_units': 256, 'num_dropout_layers': 3, 'learning_rate': 0.01}. Best is trial 1 with value: 2.641005039215088.


Epoch 1/20
32/32 [==============================] - 75s 435ms/step - loss: 38582.9922 - val_loss: 1.8028
Epoch 2/20
32/32 [==============================] - 8s 257ms/step - loss: 217.8607 - val_loss: 1.1118
Epoch 3/20
32/32 [==============================] - 8s 251ms/step - loss: 10.8777 - val_loss: 1.1162
Epoch 4/20
32/32 [==============================] - 8s 253ms/step - loss: 3.0166 - val_loss: 1.1164
Epoch 5/20
32/32 [==============================] - 8s 256ms/step - loss: 2.0570 - val_loss: 1.1164


[I 2023-03-31 15:52:23,120] Trial 4 finished with value: 1.111788272857666 and parameters: {'l2_strength': 8.67906541302328e-06, 'dropout_rate': 0.3060815370347101, 'dense_units': 512, 'num_dropout_layers': 3, 'learning_rate': 0.01}. Best is trial 4 with value: 1.111788272857666.
<ipython-input-11-1d8287abab42>:115: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.FixedTrial.suggest_float` instead.
  l2_strength = trial.suggest_loguniform("l2_strength", 1e-6, 1e-2)
<ipython-input-11-1d8287abab42>:116: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.FixedTrial.suggest_float` instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)


Best hyperparameters:  {'l2_strength': 8.67906541302328e-06, 'dropout_rate': 0.3060815370347101, 'dense_units': 512, 'num_dropout_layers': 3, 'learning_rate': 0.01}
Epoch 1/20
32/32 [==============================] - 70s 454ms/step - loss: 65241.8242 - val_loss: 2768.6770
Epoch 2/20
32/32 [==============================] - 8s 255ms/step - loss: 1216.2914 - val_loss: 1.1394
Epoch 3/20
32/32 [==============================] - 8s 251ms/step - loss: 94.4799 - val_loss: 1.1457
Epoch 4/20
32/32 [==============================] - 8s 251ms/step - loss: 11.9003 - val_loss: 1.1460
Epoch 5/20
32/32 [==============================] - 8s 255ms/step - loss: 5.9073 - val_loss: 1.1461
